# Initialize table and data from file

In [2]:
# initialize data 


###
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# plot table
import texttable as tt

def plotTable(headers,
              rows,
              valign=True,
              column_width=False,
              no_deco=False,
              precision=1):
    tab = tt.Texttable()
    if len(headers) > 0:
        tab.header(headers)
    tab.set_precision(precision)
    for row in rows:
        tab.add_row(row)

    if valign and len(headers) > 0:
        tab.set_cols_valign(['m'] * len(headers))

    if column_width:
        columns = [0] * len(rows[0])
        for row in rows:
            for i, cell in enumerate(row):
                if isinstance(row[i], str):
                    columns[i] = max(
                        columns[i],
                        max([len(line) for line in row[i].split('\n')]))
        for i, head in enumerate(headers):
            columns[i] = max(columns[i], len(head))
        tab.set_cols_width(columns)

    if no_deco:
        tab.set_deco(tt.Texttable.HEADER | tt.Texttable.VLINES)

    s = tab.draw()
    return s

def data_from_file(file_name,case):
    #case = 1-> Id from file. Iq in 0->220 > 'i_direct_flux'
    #case = 2 -> Iq from file. Id in 0->220 > 'j_q_flux'
    data = np.loadtxt(file_name)
    ids = data[0, :]    
    dfs = data[1:, :]
    iqs = list(range(220, -1, -20))
    if case == 1:
        xLabel = 'Id'
        yLabel = 'direct flux'
        Is = 'Iq'
    elif case == 2:
        xLabel = 'Iq'
        yLabel = 'quadratic flux'  
        Is = 'Id'
    return ids, dfs, iqs, xLabel, yLabel, Is


ids, dfs, iqs, xLabel, yLabel, Is = data_from_file('i_direct_flux',1 )
# plt.rcParams["figure.figsize"] = [30,20]

# PCA

In [20]:
#
import pandas as pd
import numpy as np

from scipy.interpolate import interp1d
from scipy import interpolate

from sklearn.decomposition import PCA
from sklearn import preprocessing



#INITIALIZE THE TABLE 
headers =[ 'No of PCA' ]+ ['Exaplined Variance'] + ['Mean Error'] + ['Id %s'%id for id in ids]# 
rows    = []

#create more samples
sample_size = 1000
flux = np.zeros(shape=(12, sample_size))

for curve in range(0,12): #pentru fiecare curba Iq
    y = dfs[:, curve] #iau fluxurile corespunzatoare
    x = ids #iau id-urile
        
    tck = interpolate.splrep(x, y, s=0) #face spline pe toate cele 12 puncte
    x_new = np.linspace(min(x), max(x), sample_size) #creez mai multe samples pt ids
        
    # baseline = y values obtained when interpolating with all ground truth points
    flux[curve] = interpolate.splev(x_new, tck, der=0)


#preprocess the data
df_flux = pd.DataFrame(data = flux, index = ['##   Iq %s    ##'%id for id in ids])
scaler = preprocessing.StandardScaler()
df_flux_scaled = scaler.fit_transform(df_flux)

no_of_comps = [2,25,40,50]
for comp in no_of_comps:
    row = [comp]
    
    #apply PCA
    pca = PCA(n_components=comp)
    flux_reduced = pca.fit_transform(df_flux_scaled)
    projected  = pca.inverse_transform(flux_reduced)
    projected_unscaled = scaler.inverse_transform(projected)
    df_projected = pd.DataFrame(data = projected_unscaled, index = ['##   Iq %s    ##'%id for id in ids])

    #visualize data
#     for curve in range(0,12):
#         print('curve %d no of comps %d' %(iqs[curve], comp))
#         plt.plot(x_new, flux[curve], label = 'original', color = 'black')
#         plt.plot(x_new, projected_unscaled[curve], label = 'projected')
#         plt.legend()
#         plt.grid()
#         plt.show() 
        
    row.append((pca.explained_variance_ratio_)*100)
    loss = ((df_flux - df_projected) ** 2).mean(axis = 1)
    row.append(sum(loss))
    for i in loss.values:
        row.append(i)
    rows.append(row)


# print errors in table    
print('Loss ( sum squared error * 10 ^ 4)')
table = plotTable(headers=headers, rows=rows, column_width=True, precision=5)
print(table)

Loss ( sum squared error * 10 ^ 4)
+-----------+--------------------+------------+--------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+
| No of PCA | Exaplined Variance | Mean Error | Id 0.0 | Id 20.0 | Id 40.0 | Id 60.0 | Id 80.0 | Id 100.0 | Id 120.0 | Id 140.0 | Id 160.0 | Id 180.0 | Id 200.0 | Id 220.0 |
+===========+====================+============+========+=========+=========+=========+=========+==========+==========+==========+==========+==========+==========+==========+
| 2         | [99.10929048       | 0.00000    | 0.0000 | 0.00000 | 0.00000 | 0.00000 | 0.00000 | 0.00000  | 0.00000  | 0.00000  | 0.00000  | 0.00000  | 0.00000  | 0.00000  |
|           | 0.8745198 ]        |            | 0      |         |         |         |         |          |          |          |          |          |          |          |
+-----------+--------------------+------------+--------+---------+---------+---------+---------

# Plot all Iqs curves

In [ ]:
#plt.rcParams["figure.figsize"] = [12, 8]

for idx in range(0, 12):
    df = dfs[:, idx]
    plt.plot(ids, df, label='%s = %d' %(Is,iqs[idx]))
    plt.ylim([0, 0.25])
    plt.xlim([0, 220])
    plt.xticks(ids)
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.legend(loc='best', prop={'size': 12})
plt.grid()
plt.show()

# Det Nr. Minim puncte  generatoare - Cubic spline interpolation

In [ ]:
#cubic spline interpolation

from scipy.interpolate import interp1d
from scipy import interpolate
import itertools
import sys

ids, dfs, iqs, xLabel, yLabel, Is = data_from_file('i_direct_flux', 1)

def sum_squared_error(gr, pred):
    gr = np.array(gr)
    pred = np.array(pred)
    return (10 ** 6) * (1 / gr.shape[0]) * sum((gr - pred) ** 2)

curve_indices = list(range(0, 12)) 
curves_for_plot = list(range(1, 13,2))
pts_for_plot = [6,5,4]

# try interpolating with only 10, 9, 8 ... 4, 3 points
sub_points_num  = list(range(len(ids)-1, 3, -1))

headers = ['%s' %Is] + ['pts num: %d' % pts for pts in sub_points_num] + [' ################ best_pts ########']
rows = []
x_best_5_pts = []
y_best_5_pts = []

for curve in curve_indices: #pentru fiecare curba Iq
    y = dfs[:, curve] #iau fluxurile corespunzatoare
    x = ids #iau id-urile
        
    tck = interpolate.splrep(x, y, s=0) #face spline pe toate cele 12 puncte
    x_new = np.linspace(min(x), max(x), 200)
        
    # baseline = y values obtained when interpolating with all ground truth points
    baseline = interpolate.splev(x_new, tck, der=0)
    
    row = ['%s %d' % (Is, iqs[curve])]
    best_5_points =[]
   
    for points_num in sub_points_num:
        # store best configuration
        best_pts     = (x, y)
        best_interp  = (x_new, baseline)
        best_error   = sys.float_info.max
        best_err_gr  = sys.float_info.max
    
        sub_points_sets = itertools.combinations(range(len(ids)), points_num)
        for pts in sub_points_sets:
            pts = list(pts)
            x_     = x[pts]
            y_     = y[pts]
            tck_   = interpolate.splrep(x_, y_, s=0)
            y_new_ = interpolate.splev(x_new, tck_, der=0)            
        
            # error -- difference between baseline interp curve and current one
            error = sum_squared_error(baseline, y_new_)
            
            if error < best_error:
                best_error  = error
                best_pts    = (x_, y_) # best points from combination
                best_interp = (x_new, y_new_) # best points from interpolation
                
                # interpolate in target points + evaluate error
                gr_interp = interpolate.splev(x, tck_, der=0)
                best_err_gr = sum_squared_error(y, gr_interp)
        
        if points_num == 5:
            best_5_points = best_pts[0]
            x_best_5_pts.append(best_pts[0])#to be used for linear interp
            y_best_5_pts.append(best_pts[1])#to be used for linear interp
            
        row.append(best_err_gr)
        
        if curve in curves_for_plot and points_num in pts_for_plot:
            print('%s: %d, pts num: %d' % (Is,iqs[curve], points_num))
            plt.clf()
            plt.scatter(x, y, label='orig points')
            plt.plot(x_new, baseline, label='original spline')
            plt.scatter(best_pts[0], best_pts[1], label='best pts')
            plt.plot(best_interp[0], best_interp[1], label='best interp')

            plt.ylim([0, 0.25])
            plt.xlim([0, 220])
            plt.xticks(ids)
            plt.xlabel(xLabel)
            plt.ylabel(yLabel)
            plt.legend(loc='best')
            plt.title('%s : %d, pts_num: %d' % (Is,iqs[curve], points_num))
            plt.grid()
            plt.show()
    
    row.append(best_5_points)
    rows.append(row)
    

print('Interpolation error ( sum squared error * 10 ^ 6)')
table = plotTable(headers=headers, rows=rows, column_width=True, precision=5)
print(table)


# Det. nr minim de curbe generatoare Iq  - Linear and Spline interpolation

In [ ]:
#
import sys
from scipy.interpolate import interp1d
from scipy import interpolate

best_points = [0., 40., 80., 160., 200.]
best_indices = [0,2,4,8,10] #indices of best-points
all_curves = list(range(12))
curves_for_plot = [11,6]
plt.rcParams["figure.figsize"] = [12, 8]


config_curves = [[11,0],[11,6,0],[11,8,4,0],[11,9,6,3,0],[11,8,6,4,2,0],[11,9,7,6,5,1,0]]
error_per_config = [0] * len(config_curves)


# headers = ['%s' %Is] + ['no of curves: %d' % len(nr)for nr in config_curves]+['best no of curves']+['######### best config #########']
headers = ['######### Configuration ##########'] +['   Id %d   ' % iqs[index] for index in all_curves] + ['error_per_config']
rows    = []


# flux pt best_points
def flux_for_curves_config(curves_config_indices):
    flux = [[] for i in range(12)]
    for curve in curves_config_indices:
        for idx in best_indices:
            flux[curve].append(dfs[idx][curve])
    return flux    

#incadrez curba intre doua curbe din config

def border_curves(curve, interval):
    pos = 0
    for id in interval:
        if iqs[curve] > iqs[id]:
            pos+=1
        else:
            if iqs[curve] == iqs[id]:
                if pos+1 == len(interval):
                    return pos, pos-1
                else:
                    return pos, pos+1
         
    return pos-1, pos 

def sum_squared_error(gr, pred):
    gr = np.array(gr)
    pred = np.array(pred)
    return (10 ** 6) * (1 / gr.shape[0]) * sum((gr - pred) ** 2)

for idx in range(len(config_curves)): #iau o configuratie
    indices = config_curves[idx]
    row =[[iqs[i] for i in indices]]
    plt.clf()
   
    
    for curve in all_curves: #pt fiecare curba
            
        start, end = border_curves(curve, indices)  #gasesc curbele care imi incadreaza curba de aproximat
        flux = flux_for_curves_config([start, end]) # retin fluxurile pentru curbele din config
       
        #interpolare liniara                       
        
        alpha = 1-((iqs[curve]-iqs[start])/(iqs[end]-iqs[start]))
             
        flux_approx = []
        for i in range(5):
            flux_approx.append(alpha * flux[start][i] + (1-alpha)*flux[end][i])
        
        ##Spline##
        
        #plotez cu toate cele 12 pcte
        y = dfs[:, curve] #iau fluxurile corespunzatoare
        x = ids #iau Id-urile

        tck = interpolate.splrep(x, y, s=0) #fac spline pe toate cele 12 puncte
        x_new = np.linspace(min(x), max(x), 200) #creez mai multe puncte

        # baseline = y values obtained when interpolating with all ground truth points
        baseline = interpolate.splev(x_new, tck, der=0) 
              
        # interpolate in best x points
        x_     = best_points
        y_     = flux_approx
        tck_   = interpolate.splrep(x_, y_, s=0)
        y_new_ = interpolate.splev(x_new, tck_, der=0)            
        
      # error -- difference between baseline interp curve and current one
        error = sum_squared_error(baseline, y_new_)
           
#         print('%s: %d, curves num: %d' % (Is,iqs[curve], len(indices)))
        
        if curve in curves_for_plot:
            
#             color=next(ax._get_lines.prop_cycler)['color']
            plt.scatter(x, y, label='orig points for  %s = %d' %(Is, iqs[curve]))
            plt.plot(x, y, label='original %s %d' %(Is, iqs[curve]), color = 'red')
            plt.scatter(x_, y_, label='points of spline %s = %d' %(Is, iqs[curve]))
            plt.plot(x_new, y_new_, label='spline for %s = %d' %(Is, iqs[curve]))

            plt.ylim([0, 0.25])
            plt.xlim([0, 220])
            plt.xticks(ids)
            plt.xlabel(xLabel)
            plt.ylabel(yLabel)
            plt.legend(loc='best')
            plt.title('configuration')
            plt.title([iqs[i] for i in indices])

           
        
        error_per_config[idx]+=error
        
        row.append(error)
         
    row.append(error_per_config[idx]/12)
    
    rows.append(row) 
    
     
    for i in indices:
        df = dfs[:, i]
        plt.plot(ids, df, label='Curve from config %s = %d' %(Is,iqs[i]), color = 'black')
#         plt.scatter(best_points, [dfs[j][i] for j in best_indices], color = 'black')
        plt.ylim([0, 0.25])
        plt.xlim([0, 220])
        plt.xticks(ids)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(loc='best')
    plt.grid()
    plt.show()

print('Interpolation error ( sum squared error * 10 ^ 6)')
table = plotTable(headers=headers, rows=rows, column_width=True, precision=5)
print(table)


# Curve fitting - elementary functions

In [ ]:
# # approximate de curve with different elementary functions

# def func_ord_7(x, h ,g, f, e, a, b, c, d):
#     return h * (x ** 7) + g * (x ** 6) + f * (x ** 5) + e * (x **4) + a * (x**3) + b * (x ** 2) + c * x + d

# def func_ord_6(x, g, f, e, a, b, c, d):
#     return g * (x ** 6) + f * (x ** 5) + e * (x **4) + a * (x**3) + b * (x ** 2) + c * x + d

# def func_ord_5(x, f, e, a, b, c, d):
#     return f * (x ** 5) + e * (x **4) + a * (x**3) + b * (x ** 2) + c * x + d

# def func_ord_4(x, e, a, b, c, d):
#     return e * (x **4) + a * (x**3) + b * (x ** 2) + c * x + d

# def func_ord_3(x, a, b, c, d):
#     return a * (x**3) + b * (x ** 2) + c * x + d

# def func_ord_2(x, a, b, c):
#     return a * (x**2) + b * x  + c
  
# def plot_original(iq_idx):
#     df = dfs[:, iq_idx]    
#     # plot original points     
#     plt.plot(ids, df, label = 'original')#, color = 'black')
#     plt.scatter(ids, df, color = 'black')

# def plot_curve_iq(iq_idx, func, func_descr='', label='', plot=False):
#     df = dfs[:, iq_idx]    
#     # fit curve to points     
#     w, pcov = curve_fit(func, ids, df, )
    
#     # compute error
#     df_pred = func(ids, *w)
#     error = (1 / len(ids)) * sum((df_pred - df) ** 2)
    
#     # predict new points     
#     x = np.arange(0, 240, 20)
#     y_pred = func(x, *w)
    
#     if plot:
#         plt.plot(x, y_pred,label = label)
#         plt.scatter(x, y_pred)
#         plt.title(' %s error: %.8f' % (func_descr, error))
#         plt.ylim([0, 0.25])
#         plt.xlim([0, 220])
#         plt.xticks(ids)
#         plt.xlabel(xLabel)
#         plt.ylabel(yLabel)
#         #plt.grid()
#         #plt.show()
#     return error
        
# funcs = [
#     func_ord_7,
#     func_ord_6, 
#     func_ord_5, 
#     func_ord_4, 
#     func_ord_3, 
#     func_ord_2
# ]
# dex = {
#     func_ord_7 : 'y=f(x**7)',
#     func_ord_6 : 'y=f(x**6)',
#     func_ord_5 : 'y=f(x**5)', 
#     func_ord_4 : 'y=f(x**4)',
#     func_ord_3 : 'y=f(x**3)',
#     func_ord_2 : 'y=f(x**2)',
# }

# curve_indices = list(range(0, 12)) 
# curve_for_plot = [6]

# # Get current size
# #plt.rcParams["figure.figsize"] = [12, 8]
    
# headers = ['%s ' %Is] + [dex[func] for func in funcs]
# rows    = []

# for curve_idx in curve_indices:
#     row = ['%s: %d' % (Is,iqs[curve_idx])]
#     toPlot = curve_idx in curve_for_plot
#     for func in funcs:
#         error = plot_curve_iq(curve_idx, func, '%s: %d' % (Is, iqs[curve_idx]), dex[func], plot=toPlot)
#         row.append(error * 10000)
#     rows.append(row)
        
#     if toPlot:
#         plot_original(curve_idx)
#         plt.show()
# #     plt.grid()
# #     plt.legend(loc='best')

# table = plotTable(headers=headers, rows=rows, precision=3)
# print(table)